In [ ]:
#r "nuget: Azure.Identity, 1.5.0"
#r "nuget: Microsoft.EntityFrameworkCore.SqlServer, 6.0.1"

In [ ]:
using Azure.Core;
using Azure.Identity;
using System.Collections.Generic;
using System.Data;
using System.Data.Common;
using System.Threading;
using Microsoft.Data.SqlClient;
using Microsoft.EntityFrameworkCore;
using Microsoft.EntityFrameworkCore.Diagnostics;

In [ ]:
var connectionString = "server=<YOUR_SQL_SERVER>;database=<YOUR_SQL_DATABASE>;";

In [ ]:
var errorMessage = string.Empty;
if (connectionString.Contains("<YOUR_SQL_SERVER>") && connectionString.Contains("<YOUR_SQL_DATABASE>"))
{
    errorMessage = "I hope there isn't a server called \"<YOUR_SQL_SERVER>\" and a database \"<YOUR_SQL_DATABASE>\"";
}
else if (connectionString.Contains("<YOUR_SQL_SERVER>"))
{
    errorMessage = "I hope there isn't a server called \"<YOUR_SQL_SERVER>\"";
}
else if (connectionString.Contains("<YOUR_SQL_DATABASE>"))
{
    errorMessage = "I hope there isn't a database called \"<YOUR_SQL_DATABASE>\"";
}

In [ ]:
var credential = new DefaultAzureCredential();

In [ ]:
public class SetiContext : DbContext
{
    private DbSet<TrickyDateTime> trickyDateTimes;

    public SetiContext(DbContextOptions options)
        : base(options)
    {
        this.trickyDateTimes = this.Set<TrickyDateTime>();
    }

    public async Task<DateTime> GetTrickyDateTimeUtcNowAsync(CancellationToken cancellationToken = default) 
    {
        return (await this.trickyDateTimes.FromSqlRaw("SELECT GETUTCDATE() AS [UtcNow]").SingleAsync(cancellationToken)).UtcNow;
    }

    protected override void OnModelCreating(ModelBuilder builder)
    {
        builder.Entity<TrickyDateTime>()
            .HasNoKey();
    }
}

public class TrickyDateTime
{
    public DateTime UtcNow { get; set; }
}

public class AadAuthenticationInterceptor : DbConnectionInterceptor
{
    private TokenCredential credential;
    private TokenRequestContext requestContext;
    
    public AadAuthenticationInterceptor(TokenCredential credential)
    {
        this.credential = credential;
        this.requestContext = new TokenRequestContext(new []{ "https://database.windows.net/.default" });
    }

    public override InterceptionResult ConnectionOpening(DbConnection connection, ConnectionEventData eventData, InterceptionResult result)
    {
        if (connection.State.Equals(ConnectionState.Closed))
        {
            this.InternalConnectionOpening(connection, this.credential.GetToken(this.requestContext, default));
        }

        return result;
    }

    public override async ValueTask<InterceptionResult> ConnectionOpeningAsync(DbConnection connection, ConnectionEventData eventData, InterceptionResult result, CancellationToken cancellationToken = default)
    {
        if (connection.State.Equals(ConnectionState.Closed))
        {   
            this.InternalConnectionOpening(connection, await this.credential.GetTokenAsync(this.requestContext, cancellationToken));
        }
        return result;
    }

    private void InternalConnectionOpening(DbConnection connection, AccessToken accessToken)
    {
        var sqlConnection = (SqlConnection)connection;
        sqlConnection.AccessToken = accessToken.Token;
    } 
}

In [ ]:
if(string.IsNullOrEmpty(errorMessage))
{
    Console.Out.WriteLine(await new SetiContext(new DbContextOptionsBuilder()
        .UseSqlServer(connectionString)
        .AddInterceptors(new AadAuthenticationInterceptor(credential)).Options).GetTrickyDateTimeUtcNowAsync());
}
else 
{
    Console.Error.WriteLine(errorMessage);
}